In [3]:
import pandas as pd
import numpy as np
import requests
import os

from fastprogress.fastprogress import progress_bar
from collections import Counter
from zipfile import ZipFile
from io import BytesIO


## Import data files

In [4]:
# read in gdsc1 and gdsc2 files directly from the website

def import_files():
    r = requests.get('https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.4/GDSC1_public_raw_data_24Jul22.csv.zip')
    files = ZipFile(BytesIO(r.content))
    gdsc1 = pd.read_csv(files.open('GDSC1_public_raw_data_24Jul22.csv'))

    r = requests.get('https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.4/GDSC2_public_raw_data_24Jul22.csv.zip')
    files = ZipFile(BytesIO(r.content))
    gdsc2 = pd.read_csv(files.open('GDSC2_public_raw_data_24Jul22.csv'))
    
    return gdsc1, gdsc2

gdsc1, gdsc2 = import_files()

/var/folders/nn/zyrgwx2n2kz51m6jgsh64jv00000gn/T/ipykernel_73874/111733723.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  gdsc1 = pd.read_csv(files.open('GDSC1_public_raw_data_24Jul22.csv'))


In [5]:
# combine gdsc1 and gdsc2 files into one file and save locally

def combine_data(gdsc1, gdsc2):
    
    gdsc = pd.concat([gdsc1, gdsc2])
    gdsc.drop_duplicates(inplace=True)
    
    if os.path.exists('exploratory/data') == False:
        os.makedirs('exploratory/data')
    gdsc.to_csv('exploratory/data/gdsc.csv', index=False)
    
    return gdsc

gdsc = combine_data(gdsc1, gdsc2)

## Data cleaning

In [6]:
# figure out which conditions have 3 or more replicates

# unique plate, cell, tag, drug, and concentrations
replicates = pd.DataFrame(gdsc[['DRUGSET_ID','CELL_ID','TAG','DRUG_ID','CONC']].value_counts(dropna=False))
replicates = replicates.reset_index()
replicates.rename(columns={0:'replicates'}, inplace=True)

# remove conc column and drop duplicates so it's only counting replicates for one conc
replicates.drop(['CONC'], axis=1, inplace=True)
replicates.drop_duplicates(inplace=True)

# only keep 3 or more replicates
replicates = replicates[replicates['replicates']>2]
replicates

,DRUGSET_ID,CELL_ID,TAG,DRUG_ID,replicates
0,427,8570,DMSO,4000.0,10230
1,419,8570,DMSO,4000.0,9703
2,427,8570,UN-USED,NaN,8556
3,419,8570,UN-USED,NaN,8556
4,330,7110,DMSO,4000.0,4350
...,...,...,...,...,...
395816,158,3431,L11-D5-S,1557.0,3
395817,158,3431,L11-D6-S,1557.0,3
395818,158,2786,L8-D2-S,1089.0,3
395819,159,2755,L116-D6-S,1840.0,3


In [7]:
# remove samples we don't need

replicates = replicates[replicates['TAG']!='DMSO']
replicates = replicates[replicates['TAG']!='B']
replicates = replicates[replicates['TAG']!='UN-USED']
replicates = replicates[replicates['TAG']!='NC-1']
replicates

,DRUGSET_ID,CELL_ID,TAG,DRUG_ID,replicates
1584,427,8570,PC2-D1-S,1034.0,620
1585,427,8570,PC1-D1-S,1862.0,620
1586,419,8570,PC2-D1-S,1034.0,620
1587,419,8570,PC1-D1-S,1862.0,620
2660,419,8570,T1-S,2136.0,434
...,...,...,...,...,...
395816,158,3431,L11-D5-S,1557.0,3
395817,158,3431,L11-D6-S,1557.0,3
395818,158,2786,L8-D2-S,1089.0,3
395819,159,2755,L116-D6-S,1840.0,3


In [8]:
# samples with NC-0 control

replicates_control  = replicates[replicates['TAG']=='NC-0']
replicates_control

,DRUGSET_ID,CELL_ID,TAG,DRUG_ID,replicates
7170,712_f_17,2181,NC-0,NaN,288
7171,711_a_17,3490,NC-0,NaN,288
7172,712_f_17,2878,NC-0,NaN,288
7173,712_f_17,4712,NC-0,NaN,288
7174,711_a_17,2105,NC-0,NaN,288
...,...,...,...,...,...
219575,271,2147,NC-0,NaN,6
219576,310,5513,NC-0,NaN,6
219577,217,3399,NC-0,NaN,6
219578,414_a_9,3206,NC-0,NaN,6


In [10]:
# make a list for indexing the gdsc file and dropping rows we don't need

rep_ind = []

for i in progress_bar(replicates.index):
    index = replicates.loc[i]
    rep_ind.append((index['DRUGSET_ID'], index['CELL_ID'], index['DRUG_ID'], index['TAG']))
    
len(rep_ind)

304765

In [11]:
# make a new file that just has the data in replicates (by plate, cell line, drug, and tag)

gdsc_replicates = gdsc.copy()
gdsc_replicates.set_index(['DRUGSET_ID','CELL_ID','DRUG_ID','TAG'], inplace=True)
gdsc_replicates = gdsc_replicates.loc[rep_ind]
gdsc_replicates.reset_index(inplace=True)
gdsc_replicates

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,101,2.000000,4013
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,138,2.000000,2293
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,173,2.000000,2484
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,308,2.000000,2771
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,373,2.000000,2389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872347,159,2755,1840.0,L116-D6-S,GDSC_SA,7659,6874,2015-07-02T23:00:00Z,2015-07-06T23:00:00Z,637,907044,HC-1,SIDM01073,625.0,Glo,4.0,860,0.009491,72684
2872348,159,2755,1840.0,L116-D6-S,GDSC_SA,8070,7298,2015-07-15T23:00:00Z,2015-07-19T23:00:00Z,637,907044,HC-1,SIDM01073,900.0,Glo,4.0,860,0.009491,60732
2872349,158,2786,1060.0,R2-D7-S,GDSC_SA,6617,6107,2015-06-11T23:00:00Z,2015-06-15T23:00:00Z,639,1290908,HCC-78,SIDM01068,300.0,Glo,4.0,389,0.000250,30859
2872350,158,2786,1060.0,R2-D7-S,GDSC_SA,6864,6246,2015-06-17T23:00:00Z,2015-06-21T23:00:00Z,639,1290908,HCC-78,SIDM01068,300.0,Glo,4.0,389,0.000250,34642


In [12]:
# double-check that everything has replicates

pd.DataFrame(gdsc_replicates[['CELL_ID','DRUG_ID','TAG','CONC']].value_counts(dropna=False))


0
CELL_ID DRUG_ID TAG      CONC          
2181    NaN     NC-0     NaN       3720
2878    NaN     NC-0     NaN       3594
2105    NaN     NC-0     NaN       3522
3180    NaN     NC-0     NaN       3174
8570    1034.0  PC2-D1-S 2.000000  2544
...                                 ...
2787    381.0   L16-D8-S 0.156250     3
                L16-D7-S 0.312500     3
                L16-D6-S 0.625000     3
                L16-D5-S 1.250000     3
6576    1917.0  L61-D2-S 3.162555     3

[177296 rows x 1 columns]

In [13]:
# save gdsc replicates file locally

if os.path.exists('exploratory/data') == False:
        os.makedirs('exploratory/data')

gdsc_replicates.to_csv('exploratory/data/gdsc_replicates.csv', index=False)

## Data normalization

In [15]:
# make a small test file for normalization

test = gdsc_replicates[gdsc_replicates['DRUGSET_ID']=='713_a_17']
test

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY
56008,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,906790,AGS,SIDM00850,700.0,a,3.0,26,NaN,566094
56009,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,906790,AGS,SIDM00850,700.0,a,3.0,47,NaN,699105
56010,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,906790,AGS,SIDM00850,700.0,a,3.0,50,NaN,584060
56011,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,906790,AGS,SIDM00850,700.0,a,3.0,71,NaN,730092
56012,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,906790,AGS,SIDM00850,700.0,a,3.0,74,NaN,684996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867964,713_a_17,2003,1192.0,L18-D2-S,Sanger_GDSC1,115409,57001,2013-02-14T00:00:00Z,NaN,2242,906830,C32,SIDM00890,1700.0,a,3.0,129,1.2500,238818
2867965,713_a_17,2003,1192.0,L18-D2-S,Sanger_GDSC1,115413,57002,2013-02-14T00:00:00Z,NaN,2242,906830,C32,SIDM00890,1700.0,a,3.0,129,1.2500,201052
2867966,713_a_17,2003,1192.0,L18-D3-S,Sanger_GDSC1,115401,56997,2013-02-14T00:00:00Z,NaN,2242,906830,C32,SIDM00890,1700.0,a,3.0,130,0.3125,253544
2867967,713_a_17,2003,1192.0,L18-D3-S,Sanger_GDSC1,115409,57001,2013-02-14T00:00:00Z,NaN,2242,906830,C32,SIDM00890,1700.0,a,3.0,130,0.3125,258207


In [16]:
# make a new dataframe with a normalized intensity column and test speed of different methods

# 'list_comprehension' version

test['norm_intensity'] = [(test.loc[row]['INTENSITY']) / (test[(test['CELL_ID']==test.loc[row]['CELL_ID']) & (test['DRUGSET_ID']==test.loc[row]['DRUGSET_ID']) & (test['TAG']=='NC-0')]['INTENSITY'].copy().mean()) for row in progress_bar(test.index)]


# 'for loop' version
norm = pd.DataFrame()

for plate in progress_bar(test['DRUGSET_ID'].unique()):
    df = test[test['DRUGSET_ID']==plate].copy()
    for cell in df['CELL_ID'].unique():
        df2 = df[df['CELL_ID']==cell].copy()
        df2['norm_intensity_v2'] = df2['INTENSITY'] / df2[df2['TAG']=='NC-0']['INTENSITY'].mean()
        norm = pd.concat([norm, df2])
        
norm

/var/folders/nn/zyrgwx2n2kz51m6jgsh64jv00000gn/T/ipykernel_73874/919018402.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['norm_intensity'] = [(test.loc[row]['INTENSITY']) / (test[(test['CELL_ID']==test.loc[row]['CELL_ID']) & (test['DRUGSET_ID']==test.loc[row]['DRUGSET_ID']) & (test['TAG']=='NC-0')]['INTENSITY'].copy().mean()) for row in progress_bar(test.index)]


,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,...,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity,norm_intensity_v2
56008,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,...,AGS,SIDM00850,700.0,a,3.0,26,NaN,566094,0.918598,0.918598
56009,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,...,AGS,SIDM00850,700.0,a,3.0,47,NaN,699105,1.134434,1.134434
56010,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,...,AGS,SIDM00850,700.0,a,3.0,50,NaN,584060,0.947751,0.947751
56011,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,...,AGS,SIDM00850,700.0,a,3.0,71,NaN,730092,1.184717,1.184717
56012,713_a_17,3000,NaN,NC-0,Sanger_GDSC1,115429,56938,2013-02-14T00:00:00Z,NaN,1011,...,AGS,SIDM00850,700.0,a,3.0,74,NaN,684996,1.111540,1.111540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862186,713_a_17,3201,NaN,k1,Sanger_GDSC1,115381,56991,2013-02-14T00:00:00Z,NaN,2112,...,MKN28,SIDM00260,800.0,a,3.0,374,NaN,159563,0.290901,0.290901
2862187,713_a_17,3201,NaN,k1,Sanger_GDSC1,115385,56992,2013-02-14T00:00:00Z,NaN,2112,...,MKN28,SIDM00260,800.0,a,3.0,374,NaN,144198,0.262889,0.262889
2862188,713_a_17,3201,NaN,k2,Sanger_GDSC1,115377,56990,2013-02-14T00:00:00Z,NaN,2112,...,MKN28,SIDM00260,800.0,a,3.0,375,NaN,104772,0.191011,0.191011
2862189,713_a_17,3201,NaN,k2,Sanger_GDSC1,115381,56991,2013-02-14T00:00:00Z,NaN,2112,...,MKN28,SIDM00260,800.0,a,3.0,375,NaN,119562,0.217975,0.217975


In [17]:
# 'for loop' version was much faster!

In [18]:
# try on full data

gdsc_normalized = pd.DataFrame()

for plate in progress_bar(gdsc_replicates['DRUGSET_ID'].unique()):
    df = gdsc_replicates[gdsc_replicates['DRUGSET_ID']==plate].copy()
    for cell in df['CELL_ID'].unique():
        df2 = df[df['CELL_ID']==cell].copy()
        df2['norm_intensity'] = df2['INTENSITY'] / df2[df2['TAG']=='NC-0']['INTENSITY'].mean()
        gdsc_normalized = pd.concat([gdsc_normalized, df2])
        
gdsc_normalized

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,101,2.0,4013,0.110151
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,138,2.0,2293,0.062940
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,173,2.0,2484,0.068182
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,308,2.0,2771,0.076060
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,373,2.0,2389,0.065575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275525,414_a_9,3206,NaN,NC-0,MGH_GDSC1,414_194_0004680,37809,2010-11-04T00:00:00Z,NaN,628,908141,ML-2,SIDM00441,NaN,s,3.0,35,NaN,411792,0.987130
2275526,414_a_9,3206,NaN,NC-0,MGH_GDSC1,414_194_0004680,37809,2010-11-04T00:00:00Z,NaN,628,908141,ML-2,SIDM00441,NaN,s,3.0,47,NaN,414731,0.994175
2275527,414_a_9,3206,NaN,NC-0,MGH_GDSC1,414_194_0004680,37809,2010-11-04T00:00:00Z,NaN,628,908141,ML-2,SIDM00441,NaN,s,3.0,59,NaN,403578,0.967439
2275528,414_a_9,3206,NaN,NC-0,MGH_GDSC1,414_194_0004680,37809,2010-11-04T00:00:00Z,NaN,628,908141,ML-2,SIDM00441,NaN,s,3.0,71,NaN,419658,1.005986


In [19]:
# take out the controls since we don't need them anymore for making curves

gdsc_normalized = gdsc_normalized[gdsc_normalized['TAG']!='NC-0']
gdsc_normalized

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,101,2.00000,4013,0.110151
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,138,2.00000,2293,0.062940
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,173,2.00000,2484,0.068182
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,308,2.00000,2771,0.076060
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,373,2.00000,2389,0.065575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714736,411_a_9,2046,119.0,L6-D1-S,MGH_GDSC1,411_105_0006862,39343,2011-01-14T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,74,2.00000,167767,0.381553
2714737,411_a_9,2046,119.0,L6-D1-S,MGH_GDSC1,2011-09-29 2:29:10 PM,47744,2011-09-28T23:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,74,2.00000,256431,0.583202
2714771,411_a_9,2046,106.0,L5-D9-S,MGH_GDSC1,411_105_0005823,38600,2010-12-09T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,70,0.03125,838473,1.906943
2714772,411_a_9,2046,106.0,L5-D9-S,MGH_GDSC1,411_105_0006862,39343,2011-01-14T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,70,0.03125,207531,0.471989


In [20]:
# save file locally

if os.path.exists('exploratory/data') == False:
        os.makedirs('exploratory/data')

gdsc_normalized.to_csv('exploratory/data/gdsc_normalized.csv', index=False)

## Limit to only triplicates

In [21]:
# make new dataframe to save triplicates to
triplicates = pd.DataFrame()

# unique plate, cell line, and drug
for plate in progress_bar(gdsc_normalized['DRUGSET_ID'].unique()):
    df = gdsc_normalized[gdsc_normalized['DRUGSET_ID']==plate]
    for cell in df['CELL_ID'].unique():
        df2 = df[df['CELL_ID']==cell]
        for drug in df2['DRUG_ID'].unique():
            df3 = df2[df2['DRUG_ID']==drug]
            
            # make new dataframe for numbers of replicates
            df4 = pd.DataFrame(df3[['DRUGSET_ID','CELL_ID','DRUG_ID','CONC']].value_counts())
            df4.reset_index(inplace=True)
            df4.rename(columns={0:'counts'}, inplace=True)
            
            # make sure all of the concentrations have 3 replicates
            if False not in list(df4['counts']==3):
                triplicates = pd.concat([triplicates, df3])
            
triplicates

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity
2714822,427,2419,1060.0,R2-D7-S,GDSC_005-A,31617,30392,2017-07-20T23:00:00Z,2017-07-24T23:00:00Z,2256,924187,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,19685,1.152308
2714823,427,2419,1060.0,R2-D7-S,GDSC_005-A,32440,31129,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,924187,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,17200,1.006842
2714824,427,2419,1060.0,R2-D7-S,GDSC_005-A,32441,31130,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,924187,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,23030,1.348115
2714825,427,2419,1060.0,R2-D6-S,GDSC_005-A,31617,30392,2017-07-20T23:00:00Z,2017-07-24T23:00:00Z,2256,924187,KLE,SIDM00686,300.0,Glo,4.0,1311,0.000791,21596,1.264173
2714826,427,2419,1060.0,R2-D6-S,GDSC_005-A,32440,31129,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,924187,KLE,SIDM00686,300.0,Glo,4.0,1311,0.000791,19971,1.169049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2714631,411_a_9,2046,119.0,L6-D4-S,MGH_GDSC1,411_105_0006862,39343,2011-01-14T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,77,0.250000,192338,0.437435
2714632,411_a_9,2046,119.0,L6-D4-S,MGH_GDSC1,2011-09-29 2:29:10 PM,47744,2011-09-28T23:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,77,0.250000,249452,0.567330
2714735,411_a_9,2046,119.0,L6-D1-S,MGH_GDSC1,411_105_0005823,38600,2010-12-09T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,74,2.000000,826926,1.880681
2714736,411_a_9,2046,119.0,L6-D1-S,MGH_GDSC1,411_105_0006862,39343,2011-01-14T00:00:00Z,NaN,961,909701,RPMI-6666,SIDM00813,NaN,s,3.0,74,2.000000,167767,0.381553


In [22]:
# double check that they are all triplicate

triplicates[['DRUGSET_ID','CELL_ID','DRUG_ID','CONC']].value_counts()

DRUGSET_ID  CELL_ID  DRUG_ID  CONC     
158         2786     1003.0   0.000100     3
505_a_16    2064     1016.0   0.001563     3
                     1017.0   5.000000     3
                              2.500000     3
                              1.250000     3
                                          ..
525         10629    1576.0   0.316256     3
                              0.100018     3
                              0.031631     3
                              0.010004     3
714_k_17    4740     1502.0   10.000000    3
Length: 74809, dtype: int64

In [23]:
# save locally

if os.path.exists('exploratory/data') == False:
        os.makedirs('exploratory/data')

triplicates.to_csv('exploratory/data/gdsc_triplicates.csv',index=False)

## Find samples with more than 3 replicates and break them down to groups of 3 to make more graphs

In [40]:
# repeat the process from above finding the replicates

high_count = pd.DataFrame(gdsc_normalized[['DRUGSET_ID','CELL_ID','DRUG_ID','TAG','CONC']].value_counts(dropna=False))
high_count.reset_index(inplace=True)
high_count.drop(['CONC'], axis=1, inplace=True)
high_count.drop_duplicates(inplace=True)
high_count.rename(columns={0:'counts'}, inplace=True)

# keep the ones with more than 3 replicates
high_count = high_count[high_count['counts']>3]
high_count

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,counts
0,427,8570,1034.0,PC2-D1-S,620
1,419,8570,1862.0,PC1-D1-S,620
2,427,8570,1862.0,PC1-D1-S,620
3,419,8570,1034.0,PC2-D1-S,620
4,419,8570,2136.0,T1-S,434
...,...,...,...,...,...
156775,364,2961,1594.0,L72-D3-S,4
156776,364,2961,1561.0,L83-D1-S,4
156777,311,6376,1862.0,PC1-D5-S,4
156778,311,6349,1083.0,L2-D5-S,4


In [27]:
high_ind = []

for i in progress_bar(high_count.index):
    index = high_count.loc[i]
    high_ind.append((index['DRUGSET_ID'], index['CELL_ID'], index['DRUG_ID'], index['TAG']))
    
len(high_ind)

156780

In [28]:
high_file = gdsc_normalized.copy()
high_file.set_index(['DRUGSET_ID','CELL_ID','DRUG_ID','TAG'], inplace=True)
high_file = high_file.loc[high_ind]
high_file.reset_index(inplace=True)
high_file

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,101,2.000000,4013,0.110151
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,138,2.000000,2293,0.062940
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,173,2.000000,2484,0.068182
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,308,2.000000,2771,0.076060
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,373,2.000000,2389,0.065575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524532,311,6349,1083.0,L2-D5-S,GDSC_Colo-2,20804,20280,2016-10-02T23:00:00Z,2016-10-06T23:00:00Z,154,1659928,SNU-175,SIDM00216,600.0,Glo,4.0,1334,0.156250,50908,0.998160
1524533,364,3015,1005.0,L70-D4-S,GDSC_004-B,25266,24317,2017-02-03T00:00:00Z,2017-02-07T00:00:00Z,2098,905940,K-562,SIDM00791,625.0,Glo,4.0,221,0.126524,50462,0.897286
1524534,364,3015,1005.0,L70-D4-S,GDSC_004-B,25266,24317,2017-02-03T00:00:00Z,2017-02-07T00:00:00Z,2098,905940,K-562,SIDM00791,625.0,Glo,4.0,317,0.126524,58875,1.046881
1524535,364,3015,1005.0,L70-D4-S,GDSC_004-B,25374,24468,2017-02-07T00:00:00Z,2017-02-11T00:00:00Z,2098,905940,K-562,SIDM00791,625.0,Glo,4.0,221,0.126524,56772,1.009486


In [29]:
# make all of the replicates the same number per treatment/cell line

replicates_df = pd.DataFrame()

for plate in progress_bar(high_file['DRUGSET_ID'].unique()):
    df = high_file[high_file['DRUGSET_ID']==plate]
    for cell in df['CELL_ID'].unique():
        df2 = df[df['CELL_ID']==cell]
        for drug in df2['DRUG_ID'].unique():
            df3 = df2[df2['DRUG_ID']==drug]
            df4 = pd.DataFrame(df3[['DRUGSET_ID','CELL_ID','DRUG_ID','CONC']].value_counts())
            df4.reset_index(inplace=True)
            df4.rename(columns={0:'counts'}, inplace=True)
            if len(df4) > 4:
                if len(set(df4['counts']))!=1:
                    for conc in df3['CONC'].unique():
                        df5 = df3[df3['CONC']==conc]
                        replicates_df = pd.concat([replicates_df, (df5[0:(df4['counts']).min()])])
                else:
                    replicates_df = pd.concat([replicates_df, df3])
replicates_df                        

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,101,2.00,4013,0.110151
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,138,2.00,2293,0.062940
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,173,2.00,2484,0.068182
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,308,2.00,2771,0.076060
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,753608,PC-14,SIDM00237,250.0,Glo,4.0,373,2.00,2389,0.065575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497672,405_a_9,2120,38.0,L3-D4-S,MGH_GDSC1,405_364_0010206,44076,2011-07-20T23:00:00Z,NaN,867,688086,SK-N-DZ,SIDM01100,NaN,s,3.0,41,0.25,524684,1.012615
1497677,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003277,37084,2010-09-15T23:00:00Z,NaN,867,688086,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.50,160121,0.309026
1497678,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003640,37181,2010-09-29T23:00:00Z,NaN,867,688086,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.50,222560,0.429530
1497679,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0004104,37403,2010-10-21T23:00:00Z,NaN,867,688086,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.50,551342,1.064063


In [30]:
# batch into triplicates

new_triplicates = pd.DataFrame()

for plate in progress_bar(replicates_df['DRUGSET_ID'].unique()):
    df = replicates_df[replicates_df['DRUGSET_ID']==plate]
    for cell in df['CELL_ID'].unique():
        df2 = df[df['CELL_ID']==cell]
        for drug in df2['DRUG_ID'].unique():
            df3 = df2[df2['DRUG_ID']==drug]
            df4 = pd.DataFrame(df3[['DRUGSET_ID','CELL_ID','DRUG_ID','CONC']].value_counts())
            df4.reset_index(inplace=True)
            df4.rename(columns={0:'counts'}, inplace=True)
            multiple = df4['counts'][0]//3
            batches = list(range(3, (multiple*3)+1, 3))
            for conc in df3['CONC'].unique():
                df5 = df3[df3['CONC']==conc]
                for batch in batches:
                    df6 = df5[batch-3:batch].copy()
                    df6['batch'] = batch
                    new_triplicates = pd.concat([new_triplicates, df6])
                
new_triplicates

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,...,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity,batch
0,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,...,PC-14,SIDM00237,250.0,Glo,4.0,101,2.00,4013,0.110151,3
1,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,...,PC-14,SIDM00237,250.0,Glo,4.0,138,2.00,2293,0.062940,3
2,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,...,PC-14,SIDM00237,250.0,Glo,4.0,173,2.00,2484,0.068182,3
3,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,...,PC-14,SIDM00237,250.0,Glo,4.0,308,2.00,2771,0.076060,6
4,427,8570,1034.0,PC2-D1-S,GDSC_005-A,29543,28395,2017-05-31T23:00:00Z,2017-06-04T23:00:00Z,198,...,PC-14,SIDM00237,250.0,Glo,4.0,373,2.00,2389,0.065575,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497670,405_a_9,2120,38.0,L3-D4-S,MGH_GDSC1,405_364_0003640,37181,2010-09-29T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,41,0.25,348055,0.671729,3
1497671,405_a_9,2120,38.0,L3-D4-S,MGH_GDSC1,405_364_0004104,37403,2010-10-21T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,41,0.25,568239,1.096674,3
1497677,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003277,37084,2010-09-15T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.50,160121,0.309026,3
1497678,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003640,37181,2010-09-29T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.50,222560,0.429530,3


In [31]:
pd.DataFrame(new_triplicates[['DRUGSET_ID','CELL_ID','DRUG_ID','batch','CONC']].value_counts())

0
DRUGSET_ID CELL_ID DRUG_ID batch CONC        
158        2181    1003.0  3     0.000100   3
427        3297    1862.0  6     1.000000   3
           3320    1005.0  3     0.400070   3
                                 0.126524   3
                                 0.040014   3
...                                        ..
284        6849    1054.0  9     10.000000  3
                   1057.0  3     0.000250   3
                                 0.000791   3
                                 0.002501   3
714_j_16   2015    1529.0  3     1.000000   3

[288790 rows x 1 columns]

In [35]:
all_triplicates = pd.concat([triplicates, new_triplicates])
all_triplicates

,DRUGSET_ID,CELL_ID,DRUG_ID,TAG,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,MASTER_CELL_ID,...,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,ASSAY,DURATION,POSITION,CONC,INTENSITY,norm_intensity,batch
2714822,427,2419,1060.0,R2-D7-S,GDSC_005-A,31617,30392,2017-07-20T23:00:00Z,2017-07-24T23:00:00Z,2256,...,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,19685,1.152308,NaN
2714823,427,2419,1060.0,R2-D7-S,GDSC_005-A,32440,31129,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,...,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,17200,1.006842,NaN
2714824,427,2419,1060.0,R2-D7-S,GDSC_005-A,32441,31130,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,...,KLE,SIDM00686,300.0,Glo,4.0,1312,0.000250,23030,1.348115,NaN
2714825,427,2419,1060.0,R2-D6-S,GDSC_005-A,31617,30392,2017-07-20T23:00:00Z,2017-07-24T23:00:00Z,2256,...,KLE,SIDM00686,300.0,Glo,4.0,1311,0.000791,21596,1.264173,NaN
2714826,427,2419,1060.0,R2-D6-S,GDSC_005-A,32440,31129,2017-08-09T23:00:00Z,2017-08-13T23:00:00Z,2256,...,KLE,SIDM00686,300.0,Glo,4.0,1311,0.000791,19971,1.169049,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497670,405_a_9,2120,38.0,L3-D4-S,MGH_GDSC1,405_364_0003640,37181,2010-09-29T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,41,0.250000,348055,0.671729,3.0
1497671,405_a_9,2120,38.0,L3-D4-S,MGH_GDSC1,405_364_0004104,37403,2010-10-21T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,41,0.250000,568239,1.096674,3.0
1497677,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003277,37084,2010-09-15T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.500000,160121,0.309026,3.0
1497678,405_a_9,2120,38.0,L3-D3-S,MGH_GDSC1,405_364_0003640,37181,2010-09-29T23:00:00Z,NaN,867,...,SK-N-DZ,SIDM01100,NaN,s,3.0,40,0.500000,222560,0.429530,3.0


In [33]:
if os.path.exists('exploratory/data') == False:
        os.makedirs('exploratory/data')
        
all_triplicates.to_csv('exploratory/data/all_triplicates.csv', index=False)